In [1]:
#import required packages
import requests
import json

import numpy as np
import pandas as pd

import pandas_profiling
from pandas_profiling import ProfileReport
from flatten_json import flatten
#from pandas_profiling.utils.cache import cache_file

# Building flat table from multiple tables

## Table 1: Creator Info

In [2]:
query  = """
{
    creators(first: 5, orderBy: netSalesInETH, orderDirection: desc)
    {
    id
    netSalesInETH
    netSalesPendingInETH
    netRevenueInETH
    netRevenuePendingInETH
    }
}
"""

url = 'https://api.thegraph.com/subgraphs/name/f8n/fnd'
r = requests.post(url, json={'query': query})
#print(r.status_code)
#print(r.text)

# convert json into list
json_data = json.loads(r.text)
extracted_data = json_data['data'][list(json_data['data'].keys())[0]]

#create a empty dataframe to store the extracted data
# ref to https://www.kite.com/python/answers/how-to-create-an-empty-dataframe-with-column-names-in-python
df_column_names = list(extracted_data[0].keys())
wrangled_df = pd.DataFrame(columns = df_column_names)
#print(wrangled_df.head())

# store each entry from the extracted data into the empty data frame
for i in range(len(extracted_data)):
    wrangled_df = wrangled_df.append(extracted_data[i], ignore_index = True)
    
#profile_report = ProfileReport(wrangled_df, minimal = True)
#profile_report.to_notebook_iframe()
creator_df = wrangled_df
creator_df.head()

,id,netRevenueInETH,netRevenuePendingInETH,netSalesInETH,netSalesPendingInETH
0,0xf74d1224931afa9cf12d06092c1eb1818d1e255c,1890.4,0,2224,0
1,0x4666fd1f89576e2d6fb3f2ecec5eefd3e1ba6b59,1329.178945,0,1623.6517,0
2,0x484ec62385e780f2460feac34864a77ba5a18134,513.4,0,604,0
3,0x0624d062ae9dd596de0384d37522cde46cd500d6,425,0,500,0
4,0x0f441cfad93287109f5ef834bf52f4aaaa8d8ffa,380.005263175,0,447.0650155,0


In [3]:
creator_df.describe().T

,count,unique,top,freq
id,5,5,0xf74d1224931afa9cf12d06092c1eb1818d1e255c,1
netRevenueInETH,5,5,1890.4,1
netRevenuePendingInETH,5,1,0,5
netSalesInETH,5,5,2224,1
netSalesPendingInETH,5,1,0,5


## Table 2: NFT info part I 

In [4]:
query  = """
{
    creators(first: 5, orderBy: netSalesInETH, orderDirection: desc)
    {
    nfts
        {
        id
        dateMinted
        netSalesInETH
        netSalesPendingInETH
        netRevenueInETH
        netRevenuePendingInETH
        isFirstSale
        percentSplit
        lastSalePriceInETH
        }

    }
}
"""

url = 'https://api.thegraph.com/subgraphs/name/f8n/fnd'
r = requests.post(url, json={'query': query})

# convert json into list
json_data = json.loads(r.text)
extracted_data = json_data['data'][list(json_data['data'].keys())[0]]



# store each entry from the extracted data into the empty list
wrangled_list = list()
for i in range(len(extracted_data)):
    wrangled_list.append(extracted_data[i])

    
    
# repeat above steps going 1 level deeper into the nest    
df2_column_names = list(wrangled_list[0]['nfts'][0].keys())
wrangled_df2 = pd.DataFrame(columns = df2_column_names)

for i in range(len(wrangled_list)):
    wrangled_df2 = wrangled_df2.append(wrangled_list[i]["nfts"])

nft_df1 = wrangled_df2
nft_df1.head()

,dateMinted,id,isFirstSale,lastSalePriceInETH,netRevenueInETH,netRevenuePendingInETH,netSalesInETH,netSalesPendingInETH,percentSplit
0,1618452786,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-24437,False,2224,1890.4,12677.174465,2224,14914.3229,None
0,1634633808,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-100052,False,49.5002,42.07517,158.7885,49.5002,186.81,None
1,1634808905,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-100995,False,33,3.3,0,33,0,None
2,1634809019,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-100997,True,None,0,0,0,0,None
3,1634809215,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-100998,True,None,0,0,0,0,None


In [5]:
nft_df1.describe().T

,count,unique,top,freq
dateMinted,123,123,1618452786,1
id,123,123,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-24437,1
isFirstSale,123,2,False,116
lastSalePriceInETH,116,92,2,7
netRevenueInETH,123,97,0,7
netRevenuePendingInETH,123,103,0,16
netSalesInETH,123,97,0,7
netSalesPendingInETH,123,103,0,16
percentSplit,0,0,NaN,NaN


## Table 3 NFT History

In [6]:
query  = """
{
    creators(first: 5, orderBy: netSalesInETH, orderDirection: desc)
    {
    nfts
        {     
        nftHistory
            {
            id
            event
            date
            amountInETH
            amountInTokens
            marketplace
            }
        }

    }
}
"""

url = 'https://api.thegraph.com/subgraphs/name/f8n/fnd'
r = requests.post(url, json={'query': query})

# convert json into list
json_data = json.loads(r.text)
extracted_data = json_data['data'][list(json_data['data'].keys())[0]]

# store each entry from the extracted data into the empty list
wrangled_list = list()
for i in range(len(extracted_data)):
    wrangled_list.append(extracted_data[i])

    
    
# repeat above steps going 1 level deeper into the nest    
wrangled_list2 = list()
#df2_column_names = list(wrangled_list[0]['nfts'][0].keys())
#wrangled_df2 = pd.DataFrame(columns = df2_column_names)

for i in range(len(wrangled_list)):
    wrangled_list2.append(wrangled_list[i]["nfts"])


df_column_names = list(wrangled_list2[0][0]['nftHistory'][0].keys())
wrangled_df = pd.DataFrame(columns = df_column_names)

#wrangled_df
for i in range(len(wrangled_list2[0])):
    wrangled_df = wrangled_df.append(wrangled_list2[0][i]["nftHistory"])

nfthist_df = wrangled_df
nfthist_df.head()


,amountInETH,amountInTokens,date,event,id,marketplace
0,444,None,1618595528,Bid,0x03b1c1227402f5b139f6c3d3c34ec1ffda707553a5fd...,Foundation
1,2,None,1618512578,Listed,0x0425972b4e71e3e307f0b81b9a4b4d23f559b73d46e5...,Foundation
2,122,None,1618527555,Bid,0x0708e2f65b149a1b6b429dbe366bd590786e86724c81...,Foundation
3,611,None,1618598693,Bid,0x09ed6d047d08c3c531c5ca5842028db4a1a63017f057...,Foundation
4,1337,None,1618600772,Bid,0x0d9fd52fa83ff321549aeb52fa488f69ccd95e7903a5...,Foundation


In [7]:
nfthist_df.describe().T

,count,unique,top,freq
amountInETH,33,31,2224,2
amountInTokens,0,0,NaN,NaN
date,36,36,1618595528,1
event,36,6,Bid,31
id,36,36,0x03b1c1227402f5b139f6c3d3c34ec1ffda707553a5fd...,1
marketplace,34,1,Foundation,34


## Building Connector Between Table 1 and Table 2

In [8]:
query  = """
{
    creators(first: 5, orderBy: netSalesInETH, orderDirection: desc)
    {
    id
    nfts
        {
        id
        }
    }
}
"""

url = 'https://api.thegraph.com/subgraphs/name/f8n/fnd'
r = requests.post(url, json={'query': query})

# convert json into list
json_data = json.loads(r.text)

flat_dict = flatten(json_data['data'])
flat_df = pd.DataFrame(flat_dict, index = [0])

## update the id_vars and value name here
flat_df2 = pd.melt(flat_df,id_vars = "creators_0_id", value_name = "nft")
flat_df2.pop("variable")
table12_connector = flat_df2
table12_connector.head()

,creators_0_id,nft
0,0xf74d1224931afa9cf12d06092c1eb1818d1e255c,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-24437
1,0xf74d1224931afa9cf12d06092c1eb1818d1e255c,0x4666fd1f89576e2d6fb3f2ecec5eefd3e1ba6b59
2,0xf74d1224931afa9cf12d06092c1eb1818d1e255c,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-100052
3,0xf74d1224931afa9cf12d06092c1eb1818d1e255c,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-100995
4,0xf74d1224931afa9cf12d06092c1eb1818d1e255c,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-100997


## Building Connector Between Table 2 and Table 3

In [9]:
query  = """
{
    creators(first: 5, orderBy: netSalesInETH, orderDirection: desc)
    {
    nfts
        {
        id
        nftHistory
            {
            id
            }
        }
    }
}
"""

url = 'https://api.thegraph.com/subgraphs/name/f8n/fnd'
r = requests.post(url, json={'query': query})

# convert json into list
json_data = json.loads(r.text)

flat_dict = flatten(json_data['data'])
flat_df = pd.DataFrame(flat_dict, index = [0])
flat_df

## update the id_vars and value name here
flat_df2 = pd.melt(flat_df,id_vars = "creators_0_nfts_0_id", value_name = "nftHistoryid")
flat_df2.pop("variable")
table23_connector = flat_df2
table23_connector.head()

,creators_0_nfts_0_id,nftHistoryid
0,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-24437,0x03b1c1227402f5b139f6c3d3c34ec1ffda707553a5fd...
1,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-24437,0x0425972b4e71e3e307f0b81b9a4b4d23f559b73d46e5...
2,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-24437,0x0708e2f65b149a1b6b429dbe366bd590786e86724c81...
3,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-24437,0x09ed6d047d08c3c531c5ca5842028db4a1a63017f057...
4,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-24437,0x0d9fd52fa83ff321549aeb52fa488f69ccd95e7903a5...


## Combining the tables into one single flat table

In [10]:
nft_df1_connected = pd.merge(nft_df1,table12_connector, how = "left", left_on = "id", right_on = "nft")
nft_df1_connected.pop("id")
nft_df1_connected.head()

,dateMinted,isFirstSale,lastSalePriceInETH,netRevenueInETH,netRevenuePendingInETH,netSalesInETH,netSalesPendingInETH,percentSplit,creators_0_id,nft
0,1618452786,False,2224,1890.4,12677.174465,2224,14914.3229,None,0xf74d1224931afa9cf12d06092c1eb1818d1e255c,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-24437
1,1634633808,False,49.5002,42.07517,158.7885,49.5002,186.81,None,0xf74d1224931afa9cf12d06092c1eb1818d1e255c,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-100052
2,1634808905,False,33,3.3,0,33,0,None,0xf74d1224931afa9cf12d06092c1eb1818d1e255c,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-100995
3,1634809019,True,None,0,0,0,0,None,0xf74d1224931afa9cf12d06092c1eb1818d1e255c,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-100997
4,1634809215,True,None,0,0,0,0,None,0xf74d1224931afa9cf12d06092c1eb1818d1e255c,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-100998


In [11]:
nfthist_df_connected = pd.merge(nfthist_df,table23_connector, how = "left", left_on = "id", right_on = "nftHistoryid")
nfthist_df_connected.pop("id")
nfthist_df_connected.head()

,amountInETH,amountInTokens,date,event,marketplace,creators_0_nfts_0_id,nftHistoryid
0,444,None,1618595528,Bid,Foundation,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-24437,0x03b1c1227402f5b139f6c3d3c34ec1ffda707553a5fd...
1,2,None,1618512578,Listed,Foundation,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-24437,0x0425972b4e71e3e307f0b81b9a4b4d23f559b73d46e5...
2,122,None,1618527555,Bid,Foundation,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-24437,0x0708e2f65b149a1b6b429dbe366bd590786e86724c81...
3,611,None,1618598693,Bid,Foundation,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-24437,0x09ed6d047d08c3c531c5ca5842028db4a1a63017f057...
4,1337,None,1618600772,Bid,Foundation,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-24437,0x0d9fd52fa83ff321549aeb52fa488f69ccd95e7903a5...


In [12]:
table12_connected =  pd.merge(creator_df, nft_df1_connected, how = "left", left_on = "id", right_on = "creators_0_id", suffixes = ["_creator","_nft"])
final_table = pd.merge(table12_connected, nfthist_df_connected, how = "left", left_on = "nft", right_on = "creators_0_nfts_0_id")

In [13]:
final_table.pop("creators_0_id")
final_table.pop("creators_0_nfts_0_id")
final_table.shape

(162, 20)

In [14]:
final_table.describe().T

,count,unique,top,freq
id,162,5,0xf74d1224931afa9cf12d06092c1eb1818d1e255c,158
netRevenueInETH_creator,162,5,1890.4,158
netRevenuePendingInETH_creator,162,1,0,162
netSalesInETH_creator,162,5,2224,158
netSalesPendingInETH_creator,162,1,0,162
dateMinted,158,123,1618452786,36
isFirstSale,158,2,False,151
lastSalePriceInETH,151,92,2224,36
netRevenueInETH_nft,158,97,1890.4,36
netRevenuePendingInETH_nft,158,103,12677.174465,36


In [15]:
final_table.head()

,id,netRevenueInETH_creator,netRevenuePendingInETH_creator,netSalesInETH_creator,netSalesPendingInETH_creator,dateMinted,isFirstSale,lastSalePriceInETH,netRevenueInETH_nft,netRevenuePendingInETH_nft,netSalesInETH_nft,netSalesPendingInETH_nft,percentSplit,nft,amountInETH,amountInTokens,date,event,marketplace,nftHistoryid
0,0xf74d1224931afa9cf12d06092c1eb1818d1e255c,1890.4,0,2224,0,1618452786,False,2224,1890.4,12677.174465,2224,14914.3229,None,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-24437,444,None,1618595528,Bid,Foundation,0x03b1c1227402f5b139f6c3d3c34ec1ffda707553a5fd...
1,0xf74d1224931afa9cf12d06092c1eb1818d1e255c,1890.4,0,2224,0,1618452786,False,2224,1890.4,12677.174465,2224,14914.3229,None,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-24437,2,None,1618512578,Listed,Foundation,0x0425972b4e71e3e307f0b81b9a4b4d23f559b73d46e5...
2,0xf74d1224931afa9cf12d06092c1eb1818d1e255c,1890.4,0,2224,0,1618452786,False,2224,1890.4,12677.174465,2224,14914.3229,None,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-24437,122,None,1618527555,Bid,Foundation,0x0708e2f65b149a1b6b429dbe366bd590786e86724c81...
3,0xf74d1224931afa9cf12d06092c1eb1818d1e255c,1890.4,0,2224,0,1618452786,False,2224,1890.4,12677.174465,2224,14914.3229,None,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-24437,611,None,1618598693,Bid,Foundation,0x09ed6d047d08c3c531c5ca5842028db4a1a63017f057...
4,0xf74d1224931afa9cf12d06092c1eb1818d1e255c,1890.4,0,2224,0,1618452786,False,2224,1890.4,12677.174465,2224,14914.3229,None,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-24437,1337,None,1618600772,Bid,Foundation,0x0d9fd52fa83ff321549aeb52fa488f69ccd95e7903a5...


## Conclusion

After creating the flat table, a total of 18 non null columns area available, of which 15 are non ID related.

3 additional columns on nft metadata ("name", "description", "image") are available but further data gathering is needed. (web-scraping and text processing?).

https://fnd.dev/docs/fnd-subgraph
"you can read the metadata JSON from https://ipfs.io/ipfs/${nft.tokenIPFSPath}"



In [16]:
print(creator_df.shape)
print(nft_df1.shape)
print(nft_df1_connected.shape)
print(nfthist_df.shape)
print(nfthist_df_connected.shape)

(5, 5)
(123, 9)
(123, 10)
(36, 6)
(36, 7)
